# **Yival Translator Demo** 👨‍🍳👩‍🍳
Experience Yival's powerful features by build a cheap translator with GPT-4.

Use Replicate and GPT-4's test data , you can **finetune** llama2's translation bot.

Benefit from **18x** savings while experiencing only a **6%** performance decrease.

We provide two way of yival finetune
* Replicate api call
* [SFT Trainer](https://github.com/YiVal/YiVal/blob/master/src/yival/finetune/sft_trainer.py)

With Replicate , we only use 400 examples and finetuning with 10 epochs
- bertscore-p 0.419 -> 0.445 (**+6.2%**)
- bertscore-r 0.592 -> 0.611 (**+3.2%**)
- bertscore-f 0.489 -> 0.514 (**+5.1%**)

![image](https://github.com/crazycth/pictures/assets/55043304/898a8812-bbd9-4d3d-92fa-1a8c6b433e8c)

We finetune our modal locally with [yival-sft-trainer](https://github.com/YiVal/YiVal/blob/add_conditioin_for_trainer/src/yival/finetune/sft_trainer.py) with 8 1080Ti and train for 8 hours on [EMNLP 2020 translation dataset](https://statmt.org/wmt20/translation-task.html).


For the same 400 test data sample, we achieved much better performance.
- bertscore-p 0.516 -> 0.835 (**+61.8%**)
- bertscore-r 0.653 -> 0.828 (**+27.0%**)
- bertscore-f 0.575 -> 0.831 (**+44.5%**)

Also, we see after fine-tuning, our new llama2 model can achieve **comparable performance** as gpt-3.5.

![**Model performance comparison**](https://github.com/KyleChen400/YiVal/assets/42785676/a62782d6-1523-408c-a5ce-f7069c720f68)

### **What is YiVal?**
> YiVal is a versatile platform support customize test data, evaluation methods and enhancement strategy , all in one.
It enpowers you to generate better results, reduce latency and decrease inference cost.

**~~TL~~DR**: YiVal streamlines the **evaluation** and **enhancement** of GenAI Apps, enhance ane evaluate **everything** with ease.

### **Why YiVal**


*   Native support **Multi-modal** apps: text📄 + audio🎙 + image🌃 + video🎥
*   **Multi-components**: which doesn't even have to be GenAI 😁
*   Native **RLHF** and **RLAIF** ⚙️
*   Most advanced open source **enhancement algorithms** 🪄

### **Fine-tuning LLMs in YiVal**
<img src="https://github.com/crazycth/pictures/assets/55043304/b59542de-911f-448b-9251-e054e2c71bdc" alt="image" style="max-width: 500px; max-height: 300px;">


# **Install the latest yival with git**

We provide two ways of yival install

1. install with pip
```
pip install yival
```

2. Developer Mode: The latest yival
```
git clone https://github.com/YiVal/YiVal.git
poetry config virtualenvs.create true
poetry install
```

here we install with poetry , you can find the detail below
* install poetry in colab environment
* install yival with poetry

In [ ]:
# clone the latest yival
import os
!python --version
!rm -rf YiVal
!git clone -b stable https://github.com/YiVal/YiVal.git

# install and config poetry
import shutil
!pip install poetry
POETRY_PATH = shutil.which("poetry") or (os.getenv("HOME") + "/.local/bin/poetry")
os.environ["PATH"] += os.pathsep + os.path.dirname(POETRY_PATH)
!poetry --version
!poetry config virtualenvs.create true

In [ ]:
os.chdir("/content/YiVal")
!poetry install --no-ansi

In [ ]:
!pip install pandas

# **Configure your OpenAI API key**

In yival, most of time we will use chatgpt as data_generator, evaluator , improver...

It's really amazing to find that llm is so powerful

In some of the demos provided by Yival, we use GPT-4 for data generation, result evaluation, and so on, because GPT-4 has stronger **reasoning** capabilities.


In [ ]:
os.environ['OPENAI_API_KEY'] = ''

# **Configure your [replicate](https://replicate.com/) API key**

we will use replicate to finetune llama2 for translation

In [ ]:
os.environ['REPLICATE_API_TOKEN'] = ''

# **[Optional] Change gpt-4 to gpt-3.5-turbo in config**

If you don't have a GPT-4 account, you can also use GPT-3.5-turbo to complete the entire process, you just need to modify the **model_name** in the config file.

For example , you can find `model_name` below

```yaml
description: Generate standard translation data
dataset:
  data_generators:
    openai_prompt_data_generator:
      chunk_size: 100000
      diversify: true
      model_name: gpt-4 #Change the model_name to gpt-3.5-turbo here 🦄️
      input_function:
        description:
          Given a tech startup business, generate a corresponding landing
          page headline
        name: headline_generation_for_business
        parameters:
          tech_startup_business: str
      number_of_examples: 3
      output_csv_path: generated_examples.csv
  source_type: machine_generated
```

If you want to use gpt-3.5-turbo, change the `use_gpt_35_turbo` to `True` in the below cell and run it.

It will autotimatically replace all `gpt-4` to `gpt-3.5-turbo` in all yamls provided by yival

In [ ]:
import os, glob, yaml
use_gpt_35_turbo = True  #change it to True if you don't want to use gpt-4

def replace_gpt4_recursive(data):
    if isinstance(data, str):
        return data.replace('gpt-4', 'gpt-3.5-turbo')
    elif isinstance(data, list):
        return [replace_gpt4_recursive(item) for item in data]
    elif isinstance(data, dict):
        return {key: replace_gpt4_recursive(value) for key, value in data.items()}
    else:
        return data


def replace_in_yaml_files(directory):
    for filename in glob.glob(os.path.join(directory, '*.yml')):
        with open(filename, 'r') as file:
            data = yaml.safe_load(file)
        data = replace_gpt4_recursive(data)
        with open(filename, 'w') as file:
            yaml.safe_dump(data, file)

if use_gpt_35_turbo:
  replace_in_yaml_files("/content/YiVal/demo/configs")
  print("[INFO] replace all gpt-4 to gpt-3.5-turbo. Use gpt-3.5-turbo in the coming page")
else:
  print("[INFO] use default gpt-4")

# **YiVal Finetune 🐤TuToRiAl🐤**
In this demo, we first used GPT-4 to generate some translation data and reference answers.

Then, we tested the translation BertScore of gpt-3.5-turbo and llama2-13b.

Finally, we performed finetuning on llama2 for translation using two different methods.
1. replicate api finetune
2. finetune locally with SFT-Trainer

## **Data Generator**

We will use GPT-4 to generate translation data.

Here we use openai_prompt_data_generator to generate the input of our custoom function.

```yaml
description: Generated experiment config
dataset:
  data_generators:
    openai_prompt_data_generator:
      chunk_size: 100000
      diversify: true
      # model_name specify the llm model , e.g. a16z-infra/llama-2-13b-chat:9dff94b1bed5af738655d4a7cbcdcde2bd503aa85c94334fe1f42af7f3dd5ee3
      model_name: gpt-4
      prompt:
          "Please provide a concrete and realistic test case as a dictionary for function invocation using the ** operator.
          Only include parameters, excluding description and name.
          Ensure it's succinct and well-structured.
          **Only provide the dictionary.**"
      input_function:
        description:
          The current function is to evaluate the English to Chinese translation ability of the large language model. You will play the role of a teacher, so please provide a coherent English sentence (teacher_quiz), and give the corresponding Chinese translation (teachaer_answer).
        name: translation_english_to_chinese
        parameters:
          teacher_quiz: str
          teacher_answer: str
      expected_param_name: teacher_answer
      number_of_examples: 2
      output_csv_path: standard_data.csv
      call_option:
        temperature: 1.6
        presence_penalty: 2
  source_type: machine_generated
  ```

In [ ]:
yaml_content = '''
description: Generated experiment config
dataset:
  data_generators:
    openai_prompt_data_generator:
      chunk_size: 100000
      diversify: true
      # model_name specify the llm model , e.g. a16z-infra/llama-2-13b-chat:9dff94b1bed5af738655d4a7cbcdcde2bd503aa85c94334fe1f42af7f3dd5ee3
      model_name: gpt-4
      prompt:
          "Please provide a concrete and realistic test case as a dictionary for function invocation using the ** operator.
          Only include parameters, excluding description and name.
          Ensure it's succinct and well-structured.
          **Only provide the dictionary.**"
      input_function:
        description:
          The current function is to evaluate the English to Chinese translation ability of the large language model. You will play the role of a teacher, so please provide a coherent English sentence (teacher_quiz), and give the corresponding Chinese translation (teachaer_answer).
        name: translation_english_to_chinese
        parameters:
          teacher_quiz: str
          teacher_answer: str
      expected_param_name: teacher_answer
      number_of_examples: 2
      output_csv_path: standard_data.csv
      call_option:
        temperature: 1.6
        presence_penalty: 2
  source_type: machine_generated
'''

with open('test_data_generator.yaml', 'w') as file:
    file.write(yaml_content)

# Generate data with YiVal
!poetry run yival run test_data_generator.yaml

# Visualize data
import pandas as pd
data = pd.read_csv("standard_data.csv")
data

## **Variations**

Variations are import in yival , different vairations mean different experiment environments.

Here we support two types of llm as student , and compare their capacity in translation
1. gpt-3.5-turbo
2. llama2-13b

```yaml
variations:
  - name : model_name
    variations:
      - instantiated_value: gpt-3.5-turbo
        value: gpt-3.5-turbo
        value_type: str
        variation_id: null

      - instantiated_value: replicate/a16z-infra/llama-2-13b-chat:9dff94b1bed5af738655d4a7cbcdcde2bd503aa85c94334fe1f42af7f3dd5ee3
        value: a16z-infra/llama-2-13b-chat:9dff94b1bed5af738655d4a7cbcdcde2bd503aa85c94334fe1f42af7f3dd5ee3
        value_type: str
        variation_id: null
```

## **Evaluator**

BERT-Score uses BERT's contextual embeddings to measure semantic similarity between machine and reference translations.

It's more accurate than traditional metrics as it captures semantic meaning, making it a good evaluation metric for translation tasks.

Here we use [yival-bertscore-evaluator](https://github.com/YiVal/YiVal/blob/master/src/yival/evaluators/bertscore_evaluator.py) as an evaluation metric for the model's translation ability.

```yaml
evaluators:
  - evaluator_type: individual
    name: bertscore_evaluator
    metric_calculators:
      - method: AVERAGE
    display_name: p
    indicator: p
  - evaluator_type: individual
    name: bertscore_evaluator
    metric_calculators:
      - method: AVERAGE
    display_name: r
    indicator: r
  - evaluator_type: individual
    name: bertscore_evaluator
    metric_calculators:
      - method: AVERAGE
    display_name: f
    indicator: f
```



## **YiVal Translate_Quiz**

In [ ]:
os.environ['ngrok']='true'
!poetry run ngrok config add-authtoken 2UK3G7MKgDqCqDnu36njaaE02bZ_7FqvcqBke5hbpgHjizoo7
!poetry run yival run /content/YiVal/demo/configs/translate_quiz.yml --output_path quiz100.pkl

![image](https://github.com/crazycth/pictures/assets/55043304/e6c145b9-ebbf-4445-9d8d-59adb1f9776a)

It's obvious that llama2 can't handle chinese translation really well without finetune , so next we will **finetune** llama2 to make it more powerful

# Finetune llama2 with replicate api

We provide really easy way for you to finetune llama2

the data can be source from data that fit evaluator condition , or data with expected_value generated by GPT-4

You can check our [replicate_finetune](https://github.com/YiVal/YiVal/blob/master/src/yival/dataset/replicate_finetune_utils.py) for more detail

you will get a model_name after finetune , you can call finetuned model with this model_name through replicate

In [ ]:
!poetry run python /content/YiVal/src/yival/dataset/replicate_finetune_utils.py

# Evaluate Finetuned model

Add new model_name to translate_quiz.yml
```yml
variations:
  - name : model_name
    variations:
      - instantiated_value: gpt-3.5-turbo
        value: gpt-3.5-turbo
        value_type: str
        variation_id: null

      - instantiated_value: replicate/a16z-infra/llama-2-13b-chat:9dff94b1bed5af738655d4a7cbcdcde2bd503aa85c94334fe1f42af7f3dd5ee3
        value: a16z-infra/llama-2-13b-chat:9dff94b1bed5af738655d4a7cbcdcde2bd503aa85c94334fe1f42af7f3dd5ee3
        value_type: str
        variation_id: null
      
      - instantiated_value: new_model_name
        value: new_model_name
        value_type: str
        variation_id: null
      
```

Fine-tuning with Replicate
For demo use cases, we only use **400** examples, and fine-tuning with 10 epochs, using Replicate API. Even with limited data, we saw significant performance improvement on llama2 regarding English-to-Chinese translation.
- bertscore-p 0.419 -> 0.445 (**+6.2%**)
- bertscore-r 0.592 -> 0.611 (**+3.2%**)
- bertscore-f 0.489 -> 0.514 (**+5.1%**)

![image](https://github.com/crazycth/pictures/assets/55043304/898a8812-bbd9-4d3d-92fa-1a8c6b433e8c)

In [ ]:
!poetry run yival run /content/YiVal/demo/configs/translate_quiz.yml --output_path quiz100.pkl

# Fine-tuning results locally
We also conduct a more complete fine-tuning locally with [EMNLP 2020 translation dataset](https://statmt.org/wmt20/translation-task.html).

We finetune our modal locally with [yival-sft-trainer](https://github.com/YiVal/YiVal/blob/add_conditioin_for_trainer/src/yival/finetune/sft_trainer.py) with 8 1080Ti and train for 8 hours.

For the same 400 test data sample, we achieved much better performance.
- bertscore-p 0.516 -> 0.835 (**+61.8%**)
- bertscore-r 0.653 -> 0.828 (**+27.0%**)
- bertscore-f 0.575 -> 0.831 (**+44.5%**)

Also, we see after fine-tuning, our new llama2 model can achieve **comparable performance** as gpt-3.5.

![**Model performance comparison**](https://github.com/KyleChen400/YiVal/assets/42785676/a62782d6-1523-408c-a5ce-f7069c720f68)